In [1]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = r'#path'
validation_dir = r'#pathdim = 80

In [3]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest',)
      #validation_split=0.2)
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(dim,dim),
        batch_size = 10,
        # class_mode='categorical',
        shuffle=True)
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(dim,dim),
        batch_size=10,
        # class_mode='categorical',
        shuffle=False)


Found 3581 images belonging to 36 classes.
Found 359 images belonging to 36 classes.


In [4]:
end_model = keras.Sequential([ 
      keras.layers.Flatten(), 
      keras.layers.Dense(300, activation='softmax')
])

In [5]:
inception_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(dim,dim,3))
for layers in inception_model.layers:
      layers.trainable = False

In [6]:
model = keras.Sequential([ 
      inception_model, end_model
])


model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics = (['accuracy']))

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        print("using GPU")
    except RuntimeError as e:
        print(e)

# with tf.device('/GPU:0'):
    # history = model.fit(train_generator, epochs=5, validation_data=validation_generator,batch_size=15 )

using GPU


In [8]:
from keras.layers import Input, Conv2D, Flatten, Dense, MaxPool2D
from keras.models import Model

In [11]:
input_layer = Input(shape = (dim,dim,3))
x = Conv2D(64, (5,5), activation='relu')(input_layer)
x = MaxPool2D(2,2)(x)
x_1 = Conv2D(50, (4,4), activation='relu')(x)
x_2 = Conv2D(30, (5,5), activation='relu')(x)
x_1_1 = MaxPool2D(2,2)(x_1)
x_1_2 = MaxPool2D(2,2)(x_2)
x = keras.layers.Concatenate()([x_1_1, x_1_2])
x = (Conv2D(50, (5,5), activation='relu'))(x)
x = (MaxPool2D(2,2))(x)
x_1 = Conv2D(50, (4,4), activation='relu')(x)
x_2 = Conv2D(30, (5,5), activation='relu')(x)
x_1_1 = MaxPool2D(2,2)(x_1)
x_1_2 = MaxPool2D(2,2)(x_2)
x = keras.layers.Concatenate()([x_1_1, x_1_2])
x = keras.layers.Flatten()(x)
x = Dense(36, activation='softmax')(x)

model = Model(inputs = input_layer, outputs = x)
model.compile(optimizer='Adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_100 (Conv2D)             (None, 76, 76, 64)   4864        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 38, 38, 64)   0           conv2d_100[0][0]                 
__________________________________________________________________________________________________
conv2d_101 (Conv2D)             (None, 35, 35, 50)   51250       max_pooling2d_10[0][0]           
____________________________________________________________________________________________

In [13]:
with tf.device('/GPU:0'):
    history = model.fit(train_generator, epochs=30, validation_data=validation_generator,batch_size=15 )

Epoch 1/30
359/359 [==============================] - 115s 321ms/step - loss: 2.3045 - accuracy: 0.3156 - val_loss: 1.5538 - val_accuracy: 0.4958
Epoch 2/30
359/359 [==============================] - 118s 329ms/step - loss: 2.2906 - accuracy: 0.3223 - val_loss: 1.5297 - val_accuracy: 0.5209
Epoch 3/30
359/359 [==============================] - 119s 332ms/step - loss: 2.2335 - accuracy: 0.3401 - val_loss: 1.6131 - val_accuracy: 0.5460
Epoch 4/30
359/359 [==============================] - 119s 332ms/step - loss: 2.2131 - accuracy: 0.3404 - val_loss: 1.4551 - val_accuracy: 0.5320
Epoch 5/30
359/359 [==============================] - 118s 329ms/step - loss: 2.1979 - accuracy: 0.3460 - val_loss: 1.5241 - val_accuracy: 0.5181
Epoch 6/30
359/359 [==============================] - 119s 333ms/step - loss: 2.1925 - accuracy: 0.3398 - val_loss: 1.4985 - val_accuracy: 0.5432
Epoch 7/30
359/359 [==============================] - 119s 331ms/step - loss: 2.1442 - accuracy: 0.3614 - val_loss: 1.4271 -